<a href="https://colab.research.google.com/github/saccheri1/IST_718_Lab01/blob/main/davidladdlab01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

David Ladd | IST 718 | Lab01 | 1/28/2002

In [73]:
#Load python packages that will or could be used for the project
import pandas as pd # to use panda data frame
import matplotlib.pyplot as plt #2D plotting
import numpy as np
import seaborn as sns # For plots

import plotly
import plotly.graph_objs as go
from plotly.offline import iplot
import regex

# Needed for ML and regression
%matplotlib inline

# Needed for building models
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split

# example of training a final regression model
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

import statsmodels.api as sm
import statsmodels.formula.api as smf

Load coaches9 and college football stadium datasets and do some initial cleansing.

In [65]:
# Load the dataset and create a data frame
coachesdf = pd.read_csv('https://raw.githubusercontent.com/2SUBDA/IST_718/master/Coaches9.csv')

# Inspect data frame
coachesdf.head()

,School,Conference,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout
0,Air Force,Mt. West,Troy Calhoun,885000,885000,247000,--,$0,--
1,Akron,MAC,Terry Bowden,"$411,000","$412,500","$225,000","$50,000",$0,"$688,500"
2,Alabama,SEC,Nick Saban,"$8,307,000","$8,307,000","$1,100,000","$500,000",$0,"$33,600,000"
3,Alabama at Birmingham,C-USA,Bill Clark,"$900,000","$900,000","$950,000","$165,471",$0,"$3,847,500"
4,Appalachian State,Sun Belt,Scott Satterfield,"$712,500","$712,500","$295,000","$145,000",$0,"$2,160,417"


In [66]:
#Upload College Football Stadium Comparative Data, create data frame and view head
#Source: https://www.collegegridirons.com/comparisons-by-capacity/
stadiumsdf = pd.read_csv('https://raw.githubusercontent.com/saccheri1/IST_718_Lab01/main/stadiums.csv')
print(stadiumsdf.head())

            Stadium     College Conference  Capacity  YearOpened  StadiumAge
0  Michigan Stadium    Michigan    Big Ten    107601        1927          95
1    Beaver Stadium  Penn State    Big Ten    106572        1960          62
2      Ohio Stadium  Ohio State    Big Ten    104944        1922         100
3        Kyle Field   Texas A&M        SEC    102733        1904         118
4   Neyland Stadium   Tennessee        SEC    102521        1921         101


In [67]:
coachesdf.rename(columns={'School':'College'}, inplace = True)
coachesdf.columns

Index(['College', 'Conference', 'Coach', 'SchoolPay', 'TotalPay', 'Bonus',
       'BonusPaid', 'AssistantPay', 'Buyout'],
      dtype='object')

In [69]:
#Concatenate coachesdf and stadiumsdf data frames into newdf data frame and inspect
left = stadiumsdf
right = coachesdf
newdf = pd.merge(left,right, how = 'outer', on='College')
display(newdf)


,Stadium,College,Conference_x,Capacity,YearOpened,StadiumAge,Conference_y,Coach,SchoolPay,TotalPay,Bonus,BonusPaid,AssistantPay,Buyout
0,Michigan Stadium,Michigan,Big Ten,107601.0,1927.0,95.0,Big Ten,Jim Harbaugh,"$7,504,000","$7,504,000","$1,325,000","$150,000",$0,"$17,111,110"
1,Beaver Stadium,Penn State,Big Ten,106572.0,1960.0,62.0,Big Ten,James Franklin,"$4,800,000","$4,800,000","$1,000,000","$300,000",$0,"$18,375,000"
2,Ohio Stadium,Ohio State,Big Ten,104944.0,1922.0,100.0,Big Ten,Urban Meyer,"$7,600,000","$7,600,000","$775,000","$350,000",$0,"$38,058,402"
3,Kyle Field,Texas A&M,SEC,102733.0,1904.0,118.0,SEC,Jimbo Fisher,"$7,500,000","$7,500,000","$1,350,000",--,$0,"$68,125,000"
4,Neyland Stadium,Tennessee,SEC,102521.0,1921.0,101.0,SEC,Jeremy Pruitt,"$3,846,000","$3,846,000","$1,200,000",--,$0,"$11,780,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,NaN,Southern Methodist,NaN,NaN,NaN,NaN,AAC,Sonny Dykes,--,--,--,--,$0,--
142,NaN,Southern Mississippi,NaN,NaN,NaN,NaN,C-USA,Jay Hopson,"$500,000","$500,000","$870,000","$15,000",$0,"$1,583,333"
143,NaN,Texas Christian,NaN,NaN,NaN,NaN,Big 12,Gary Patterson,"$4,840,717","$4,840,717",--,--,$0,--
144,NaN,Texas-El Paso,NaN,NaN,NaN,NaN,C-USA,Dana Dimel,"$700,000","$700,000","$741,665",--,$0,"$2,991,667"


In [94]:
#Obliterate unneeded variables, relabel Conference_y to Conference, and relabel "Capacity" to "stadiumCapacity"
x=newdf.drop(['Stadium','Conference_x', 'SchoolPay','BonusPaid','Bonus', 'AssistantPay','Buyout'], axis=1)
x.rename(columns={'Conference_y':'Conference'}, inplace = True)
x.rename(columns={'Capacity':'StadiumCapacity'}, inplace = True)
#Obliterate records with NA's, create new dataframe, and inspect results
booyahdf = x.dropna()
display(booyahdf)
booyahdf.describe()

,College,StadiumCapacity,YearOpened,StadiumAge,Conference,Coach,TotalPay
0,Michigan,107601.0,1927.0,95.0,Big Ten,Jim Harbaugh,"$7,504,000"
1,Penn State,106572.0,1960.0,62.0,Big Ten,James Franklin,"$4,800,000"
2,Ohio State,104944.0,1922.0,100.0,Big Ten,Urban Meyer,"$7,600,000"
3,Texas A&M,102733.0,1904.0,118.0,SEC,Jimbo Fisher,"$7,500,000"
4,Tennessee,102521.0,1921.0,101.0,SEC,Jeremy Pruitt,"$3,846,000"
...,...,...,...,...,...,...,...
123,Georgia State,23000.0,1996.0,26.0,Sun Belt,Shawn Elliott,"$569,000"
124,Ball State,22500.0,1967.0,55.0,MAC,Mike Neu,"$435,689"
125,Western Kentucky,22113.0,1968.0,54.0,C-USA,Mike Sanford Jr.,"$805,850"
126,Old Dominion,20118.0,1936.0,86.0,C-USA,Bobby Wilder,"$654,667"


,StadiumCapacity,YearOpened,StadiumAge
count,114.000000,114.000000,114.000000
mean,51841.736842,1956.210526,65.789474
std,23401.052546,30.159304,30.159304
min,15000.000000,1904.000000,5.000000
25%,30662.500000,1928.250000,47.000000
50%,49625.000000,1958.500000,63.500000
75%,63773.500000,1975.000000,93.750000
max,107601.000000,2017.000000,118.000000


Explore the booyah data frame. 

In [95]:
booyahdf.TotalPay.apply(lambda x: x.replace('$',''))


0      7,504,000 
1      4,800,000 
2      7,600,000 
3      7,500,000 
4      3,846,000 
          ...    
123      569,000 
124      435,689 
125      805,850 
126      654,667 
130      400,000 
Name: TotalPay, Length: 114, dtype: object

In [96]:
booyahdf['TotalPay'] = pd.to_numeric(booyahdf['TotalPay'],errors = 'coerce')
cols = ['TotalPay']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [97]:
booyahdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114 entries, 0 to 130
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   College          114 non-null    object 
 1   StadiumCapacity  114 non-null    float64
 2   YearOpened       114 non-null    float64
 3   StadiumAge       114 non-null    float64
 4   Conference       114 non-null    object 
 5   Coach            114 non-null    object 
 6   TotalPay         2 non-null      float64
dtypes: float64(4), object(3)
memory usage: 7.1+ KB


In [99]:
booyahdf.describe()

,StadiumCapacity,YearOpened,StadiumAge,TotalPay
count,114.000000,114.000000,114.000000,2.000000
mean,51841.736842,1956.210526,65.789474,908760.500000
std,23401.052546,30.159304,30.159304,33602.421349
min,15000.000000,1904.000000,5.000000,885000.000000
25%,30662.500000,1928.250000,47.000000,896880.250000
50%,49625.000000,1958.500000,63.500000,908760.500000
75%,63773.500000,1975.000000,93.750000,920640.750000
max,107601.000000,2017.000000,118.000000,932521.000000
